In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os 
import numpy as np
import cv2
# for data augmentation 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# train_dir = 'data/train'
# val_dir = 'data/test'

In [ ]:
PARENT_DIR = "content/drive/MyDrive/ChestXRays"

train_dir = PARENT_DIR + "/train"
val_dir = PARENT_DIR + "/validate"
test_dir = PARENT_DIR + "/test"

CATEGORIES = ["NORMAL", "PNEUMONIA"]
IMG_SIZE_X = 400
IMG_SIZE_Y = 400

In [ ]:
def show_img(image):
    plt.imshow(image, cmap = plt.cm.gray)
    print("size: ", image.shape)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255) # 0 - 255 to 0 - 1
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(IMG_SIZE_X,IMG_SIZE_Y), # image size
        batch_size=64,
        color_mode="grayscale", 
        class_mode='categorical' # type of label array return
)

validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(IMG_SIZE_X,IMG_SIZE_Y),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

testing_generator = val_datagen.flow_from_directory(
        test_dir,
        target_size=(IMG_SIZE_X,IMG_SIZE_Y),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 5232 images belonging to 2 classes.
Found 347 images belonging to 2 classes.
Found 277 images belonging to 2 classes.


In [ ]:
# can be due to less no of neurons
# regularization (L1, L2)
# other arch 
  # resnet
  # mass rcnn
  # rcnn

model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(filters=32,kernel_size=(4,4), activation = "relu", input_shape=(IMG_SIZE_X,IMG_SIZE_Y,1)) ,
        tf.keras.layers.MaxPooling2D(pool_size = (2,2)), # reduce spatial resolution

        tf.keras.layers.Conv2D(filters=64,kernel_size=(4,4), activation = "relu") ,
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)), 

        tf.keras.layers.Flatten(), # size will be selected aumatically
        tf.keras.layers.Dense(64, activation = "relu"),

        # softmax gives prob dist sum = 1 and typically used for multivariate classification
        tf.keras.layers.Dense(2, activation = "softmax"), # classifying two categories only
])

model.compile(
    optimizer="adam",
#               optimizer='SGD',
#               optimizer='rmsprop',
    loss = "categorical_crossentropy",
    metrics = ["accuracy"]
)

In [ ]:
# x = train_generator.next()
model.load_weights(PARENT_DIR + "/model.h5")
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 397, 397, 32)      544       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 198, 198, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 195, 195, 64)      32832     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 97, 97, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 602176)            0         
_________________________________________________________________
dense (Dense)                (None, 64)                38539328  
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 1

In [ ]:
# model.fit(X,y, epochs = 1)

model.fit(
        train_generator,
        steps_per_epoch=5232  // 64, # 28709 training images
        epochs=1,
        validation_data=validation_generator,
        validation_steps=624 // 64
        )

model.save_weights('/content/drive/MyDrive/ChestXRays/model.h5')
 

81/81 [==============================] - 1633s 20s/step - loss: 0.1975 - accuracy: 0.9226 - val_loss: 0.2317 - val_accuracy: 0.9070


In [ ]:
# model.fit(X,y, epochs = 1)

model.fit(
        train_generator,
        steps_per_epoch=5232  // 64, # 28709 training images
        epochs=1,
        validation_data=validation_generator,
        validation_steps=624 // 64
        )

model.save_weights('/content/drive/MyDrive/ChestXRays/model.h5')

81/81 [==============================] - 1638s 20s/step - loss: 0.0918 - accuracy: 0.9667 - val_loss: 0.5070 - val_accuracy: 0.8106


In [ ]:
model.evaluate(testing_generator)

5/5 [==============================] - 96s 23s/step - loss: 1.4563 - accuracy: 0.6663


[1.6449230909347534, 0.6389891505241394]

In [ ]:
model.evaluate(validation_generator)

6/6 [==============================] - 133s 26s/step - loss: 0.4410 - accuracy: 0.8357


[0.44100138545036316, 0.8357348442077637]